In [19]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.semi_supervised import LabelPropagation
from sklearn.model_selection import train_test_split

In [20]:
df = pd.read_csv('data.csv')
df.head()

,track_id,track_name,playlist_id,danceability,energy,acousticness,valence,tempo
0,2I0G8OVqgjLvY5fBC9Qc0J,Mountains and Molehills - feat. Turin Brakes,0,0.595,0.763,0.010500,0.0380,139.983
1,295nnPutAyvfXpVsfwJbga,Sweet Shop,0,0.592,0.886,0.004870,0.1050,139.913
2,3k3DZvTCSD4HKvj26dQDk8,Jump Ya Body - Dub Mix,0,0.734,0.969,0.037200,0.2260,140.024
3,2XbF32Nwvi5Kz7BRR4cIGz,Thundergun,0,0.527,0.846,0.000048,0.0631,140.008
4,5gaZEkMg6D9QuOnmAVpYVX,The Great Divide - Seven Lions Dub Mix,0,0.490,0.739,0.002010,0.1840,140.019


In [43]:
data = df.values
propagated, unlabeled = train_test_split(data, test_size=30)

In [22]:
unlabeled_df = pd.DataFrame(unlabeled)
unlabeled_df.columns = df.columns
unlabeled_df

,track_id,track_name,playlist_id,danceability,energy,acousticness,valence,tempo
0,6nmz4imkDcmtwMjocAzFSx,Hotline Bling,18,0.896,0.623,0.00257,0.564,134.962
1,5yH1lUTtIVIQAtKpvIEM5J,Ghosts - Extended Mix,15,0.574,0.875,0.000351,0.516,127.994
2,3xIwnKhZxWxKcecqrfXBJj,The Drop,14,0.689,0.879,0.104,0.544,170.036
3,35Zv0HFPaXFfkZg32R84OH,Sweet Nothing - Diplo + Grandtheft Remix,23,0.401,0.956,0.308,0.25,140.058
4,6TVhP2a7ESba4pR5ePflRw,The Anthem,7,0.791,0.782,0.00446,0.902,123.975
5,3CeYdUfGPCjKMDYyI1PpCh,Otherside,3,0.458,0.795,0.00316,0.493,123.229
6,1CvCdoP0Dqx8qrdt2ZdMH6,When I'm Gone - Album Version (Edited),37,0.64,0.736,0.0502,0.739,75.159
7,0DIekYoiwwr1kkTUGmQlSL,Strobe - Radio Edit,3,0.656,0.662,8.12e-05,0.0748,128.022
8,2KlZexJjJPuNWcN5uAG1GU,Gold (feat. Yuna),2,0.479,0.675,0.0319,0.18,145.022
9,2Aly0kBHnpKvRiKi0qSaNR,Bad Machine (feat. Insomnia) - Original Mix,0,0.606,0.824,0.000799,0.43,140.076


## Moods:
    0: Depressed
    1: Excited
    2: Aggressive
    3: Relaxing
    4: Cheerful
    5: Silly
    6: Triumphant

In [24]:
labels = []
for song in unlabeled_df['track_name']:
    labels.append(input(song + ': '))

Hotline Bling: 0
Ghosts - Extended Mix: 4
The Drop: 2
Sweet Nothing - Diplo + Grandtheft Remix: 1
The Anthem: 6
Otherside: 3
When I'm Gone - Album Version (Edited): 0
Strobe - Radio Edit: 4
Gold (feat. Yuna): 3
Bad Machine (feat. Insomnia) - Original Mix: 2
E.T. - Benny Benassi Radio Edit: 3
I Remember - Strobelite Mix: 3
This Is What It Feels Like: 4
Doomsday: 1
Psychedelic Stereo: 1
In The Ayer (feat. will.I.am): 6
Check Yes, Juliet: 1
Interplanetary - Original Mix: 3
Whatcha Say: 4
Midnight Terrain: 1
I Will Never Let You Down: 4
I Need Your Love: 3
Heartbeat (DMNDZ Remix) [feat. Collin McLoughlin]: 1
Smoke Filled Room: 3
Supernova: 1
Shotgun: 2
Legacy - Radio Edit: 2
Really Got It: 2
Marvins Room: 0
Dímelo: 2


In [37]:
temp_df = pd.DataFrame(labels)
temp_df.columns = ['mood']
final_df = pd.concat([unlabeled_df, temp_df], axis=1)
final_df

,track_id,track_name,playlist_id,danceability,energy,acousticness,valence,tempo,mood
0,6nmz4imkDcmtwMjocAzFSx,Hotline Bling,18,0.896,0.623,0.00257,0.564,134.962,0
1,5yH1lUTtIVIQAtKpvIEM5J,Ghosts - Extended Mix,15,0.574,0.875,0.000351,0.516,127.994,4
2,3xIwnKhZxWxKcecqrfXBJj,The Drop,14,0.689,0.879,0.104,0.544,170.036,2
3,35Zv0HFPaXFfkZg32R84OH,Sweet Nothing - Diplo + Grandtheft Remix,23,0.401,0.956,0.308,0.25,140.058,1
4,6TVhP2a7ESba4pR5ePflRw,The Anthem,7,0.791,0.782,0.00446,0.902,123.975,6
5,3CeYdUfGPCjKMDYyI1PpCh,Otherside,3,0.458,0.795,0.00316,0.493,123.229,3
6,1CvCdoP0Dqx8qrdt2ZdMH6,When I'm Gone - Album Version (Edited),37,0.64,0.736,0.0502,0.739,75.159,0
7,0DIekYoiwwr1kkTUGmQlSL,Strobe - Radio Edit,3,0.656,0.662,8.12e-05,0.0748,128.022,4
8,2KlZexJjJPuNWcN5uAG1GU,Gold (feat. Yuna),2,0.479,0.675,0.0319,0.18,145.022,3
9,2Aly0kBHnpKvRiKi0qSaNR,Bad Machine (feat. Insomnia) - Original Mix,0,0.606,0.824,0.000799,0.43,140.076,2


In [45]:
fit_X = final_df.iloc[:, 3:8].values
fit_y = final_df['mood'].values
lp = LabelPropagation()
lp.fit(fit_X, fit_y)
lp.predict(propagated[:, 3:])

array(['0', '1', '2', '3', '0', '0', '0', '1', '3', '2', '3', '1', '3',
       '1', '0', '1', '1', '3', '2', '0', '0', '2', '2', '1', '0', '1',
       '0', '0', '1', '1', '1', '1', '6', '4', '3', '0', '4', '2', '3',
       '3', '1', '1', '1', '0', '3', '2', '1', '0', '1', '1', '1', '3',
       '3', '0', '1', '6', '1', '1', '1', '1', '1', '3', '0', '2', '0',
       '1', '1', '2', '1', '1', '4', '1', '4', '6', '1', '3', '6', '0',
       '3', '3', '1', '1', '1', '3', '4', '3', '3', '0', '2', '0', '3',
       '2', '1', '1', '4', '1', '6', '4', '6', '1', '2', '0', '0', '1',
       '2', '1', '1', '1', '2', '0', '4', '0', '2', '1', '1', '4', '1',
       '2', '6', '0', '3', '2', '1', '6', '1', '1', '3', '3', '1', '1',
       '1', '3', '1', '0', '1', '3', '6', '4', '0', '1', '0', '0', '0',
       '3', '3', '1', '1', '3', '3', '1', '2', '1', '2', '2', '1', '0',
       '1', '1', '1', '4', '3', '4', '2', '2', '3', '0', '3', '1', '4',
       '1', '1', '1', '2', '1', '1', '6', '4', '2', '3', '4', '3